In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar os eventos e cursos por lá.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber informações sobre novos cursos, eventos e promoções.
*   **Verificar a página de eventos da Alura:** Acesse regularmente a página de eventos no site da Alura para ver se a Imersão IA com Google Gemini está programada.
*   **Pesquisar no site da Alura:** Utilize a ferramenta de busca no site da Alura procurando por "Imersão IA Google Gemini".

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. O curso é gratuito e totalmente online, sem pré-requisitos. As inscrições podem ser feitas até o dia 11 de maio no site da Alura.

A Imersão IA é um curso que visa ensinar aos alunos como usar a inteligência artificial para otimizar o trabalho e transformar ideias em soluções reais. O curso é dividido em cinco aulas online, onde os alunos aprenderão a criar prompts poderosos, automatizar tarefas e desenvolver um sistema de agentes autônomos. Ao final do curso, os alunos receberão um certificado Alura + Google.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        #model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Vamos então criar o post sobre novidades em agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Tudo bem, vou procurar os lançamentos mais relevantes e recentes sobre agentes de IA, com foco nos últimos 30 dias e priorizando aqueles com maior cobertura noticiosa e entusiasmo.
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e tendências de agentes de IA que parecem ser os mais relevantes e comentados até o momento em maio de 2025:
> 
> 1.  **Expansão e Adoção de Agentes de IA em Várias Indústrias**: Agentes de IA estão remodelando indústrias como governo, saúde e finanças, melhorando a eficiência operacional, aprimorando a experiência do cliente e reduzindo custos. A expectativa é que, até 2028, um terço dos aplicativos de software corporativo incluirão IA Agentic, permitindo que 15% das decisões de trabalho diárias sejam tomadas de forma autônoma.
> 2.  **Novos Lançamentos e Atualizações de Grandes Empresas**: Empresas como IBM, Anthropic, Amazon, Google e OpenAI anunciaram atualizações e lançamentos significativos que prometem revolucionar o cenário da IA, com ferramentas inovadoras e integração avançada entre agentes. A IBM, por exemplo, lançou novas ferramentas integradas ao watsonx Orchestrate, oferecendo soluções para integrar, customizar e implantar agentes, além de disponibilizar agentes pré-construídos para áreas como RH, vendas e compras.
> 3.  **Agentes de IA no Comércio e Varejo**: Um relatório da Visa e do Institute for the Future (IFTF) indica que os agentes de IA vão alterar drasticamente a forma como compramos.  82% dos brasileiros expressam interesse em assistentes pessoais com tecnologia de IA, o que demonstra uma alta aceitação dessa tecnologia no país.
> 4.  **iFood Implementa Agentes de IA da Salesforce**: O iFood está utilizando agentes de IA da Salesforce para automatizar tarefas internas, com planos de expandir para 2.000 agentes em breve. Um agente autônomo focado no atendimento da vertical de benefícios da companhia será lançado em breve, com o objetivo de agilizar o tempo de resposta aos clientes do iFood Card.
> 5.  **Frameworks e Ferramentas para Construir Agentes de IA**: Estão surgindo diversas ferramentas e frameworks para simplificar o desenvolvimento de agentes de IA, como LangGraph (Python), construído sobre o ecossistema LangChain. Modelos de linguagem como GPT-4 (OpenAI), Claude (Anthropic) e Llama 3, além de APIs de motores de busca e ferramentas de processamento de dados, são componentes essenciais para a construção de agentes de IA.
> 
> Em resumo, o campo dos agentes de IA está passando por um rápido desenvolvimento, com lançamentos de novas ferramentas e plataformas, bem como a crescente adoção em diversos setores.
> 


--------------------------------------------------------------


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-35' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho


--- 📝 Resultado do Agente 2 (Planejador) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚀 **Agentes de IA: A Revolução Silenciosa nas Empresas Brasileiras Chegou!**
> 
> Você já parou pra pensar em como a inteligência artificial está mudando o dia a dia das empresas aqui no Brasil? Não estamos falando só de chatbots, mas sim de Agentes de IA - sistemas inteligentes que agem de forma mais proativa!
> 
> 📊 A boa notícia? A adoção está acelerando! Quase um quarto das empresas brasileiras já embarcou na onda da IA. E quem investe em IA generativa está vendo a mágica acontecer:
> 
> ✅ **+14% de produtividade**
> ✅ **+9% nos resultados financeiros**
> 
> Essa IA não é mais coisa de filme! Ela está ajudando em diversas áreas, desde otimizar processos no RH e finanças até acelerar o desenvolvimento de software e criar campanhas de marketing mais eficazes.
> 
> Claro, a jornada tem seus desafios – como a necessidade de infraestrutura e, principalmente, de talentos qualificados para trabalhar com essas tecnologias. Mas a transformação na gestão e eficiência operacional é inegável.
> 
> O futuro do trabalho está sendo redesenhado pelos Agentes de IA. E a sua empresa, e você, estão prontos para essa mudança?
> 
> #Tecnologia #InteligenciaArtificial #AgentesDeIA #Inovacao


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
